In [42]:
# recommender function
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler


In [43]:
movies_original = pd.read_csv('movies.csv')
ratings_original = pd.read_csv('ratings.csv')
links_original = pd.read_csv('links.csv')

In [44]:
ratings = ratings_original.copy()
ratings.sample(3)

,userId,movieId,rating,timestamp
57755,380,52722,3.0,1494690900
100007,610,7147,4.0,1493845044
59183,386,160,3.0,842610344


In [45]:
ratings.loc[ratings_original['userId'] ==610]
# user item matrix 
users_crosstab_original = ratings.pivot_table(
    index='userId', columns='movieId', values='rating')


# values=None, index=None, columns=None, aggfunc='mean', fill_value=None,
#                    margins=False, dropna=True, margins_name='All', observed=False, sort=True)


users_crosstab_original.fillna(0, inplace=True)


In [46]:
users_crosstab = users_crosstab_original.copy()


In [55]:
def user_based_recommender(user_id, n):
        movies = movies_original.copy()
        ratings = ratings_original.copy()
        
        if len(ratings.loc[ratings['userId']== user_id]):
            # user based approach using cosine similarity
           
            # user item matrix
            users_crosstab_original = ratings.pivot_table(
    index='userId', columns='movieId', values='rating')
            users_crosstab_original.fillna(0, inplace=True)
            users_crosstab = users_crosstab_original.copy()
            # user similarity matrix
            user_similarity = cosine_similarity(users_crosstab)
            # turn similarities to weights
            user_similarity = MinMaxScaler().fit_transform(user_similarity)
            # estimate the missing ratings
            user_predicted_ratings = np.dot(user_similarity, users_crosstab)
            # turn the predicted ratings to a dataframe
            user_predicted_ratings = pd.DataFrame(
                user_predicted_ratings, index=users_crosstab.index, columns=users_crosstab.columns)
            # get the movies that the user has already rated
            user_rated = users_crosstab.loc[user_id, :]
            # get the movies that the user has not rated
            user_not_rated = user_rated[user_rated == 0]
            # get the movies that the user has not rated and sort them by the predicted ratings
            user_predicted_ratings = user_predicted_ratings.loc[user_id, user_not_rated.index]
            user_predicted_ratings = user_predicted_ratings.sort_values(    ascending=False)
            # get the top n movies
            user_predicted_ratings = user_predicted_ratings.head(n)
            # get movie name
            movieslist = movies.loc[movies['movieId'].isin(user_predicted_ratings.index), ['movieId', 'title']]
            
            # similarity = pd.DataFrame(user_similarity, index=users_crosstab.index, columns=users_crosstab.index)
            return movieslist
        else:
            
            return "Movie not found"
    
        


user_based_recommender(610, 4)


,movieId,title
322,364,"Lion King, The (1994)"
1183,1580,Men in Black (a.k.a. MIB) (1997)
1284,1704,Good Will Hunting (1997)
1734,2329,American History X (1998)
